# Tema 2

Topics:
1. Metoda substitutiei inverse
2. Algoritmul de Elminiare Gauss

In [361]:
import numpy as np
import unittest
import random

np.random.seed(10)

# functions that need to be implemented
from tema2 import substitution_method#, get_inv, gauss_algorithm

# dummy functions that mirror standard methods from libraries (but you'll have to discover them)
# from tema2 import get_norm, lib_solve, lib_inv

# from tema2 import plot_matrix_evolution

np.set_printoptions(suppress=True)

## 1. Metoda substitutiei inverse

**TL;DR:** Metoda substituiei inverse rezolva sisteme linare in care matricea asociata sistemului este triunghiulara (inferior sau superior).

* **Ce este o matrice triunghiulara superior?** O matrice triunghiulara superior este o matrice care are valori nule sub diagonala principala. 

In [362]:
# generate a 4x4 matrix with elements from 1 to 16
A = (np.arange(4*4) + 1).reshape(4, 4)
A

array([[ 1,  2,  3,  4],
       [ 5,  6,  7,  8],
       [ 9, 10, 11, 12],
       [13, 14, 15, 16]])

In [363]:
# generated superior triangular matrix from A
np.triu(A)

array([[ 1,  2,  3,  4],
       [ 0,  6,  7,  8],
       [ 0,  0, 11, 12],
       [ 0,  0,  0, 16]])

* **Ce este o matrice triunghiulara inferior?** O matrice triunghiulara inferior este o matrice care are valori nule deasupra diagonala principala. 

In [364]:
# generated inferior triangular matrix from A
np.tril(A)

array([[ 1,  0,  0,  0],
       [ 5,  6,  0,  0],
       [ 9, 10, 11,  0],
       [13, 14, 15, 16]])

### Metoda substitutiei inverse
Fie sistemul liniar:
$$
A x=b
$$

unde matricea sistemului $A$ este **triunghiulară superior**. Pentru a găsi soluţia unică a sistemului, trebuie ca matricea să fie nesingulară. Determinantul matricelor triunghiulare este dat de formula:
$$
\operatorname{det} A=a_{11} a_{22} \cdots a_{n n}
$$
Prin urmare, pentru rezolvarea sistemului vom presupune că:
$$
\operatorname{det} A \neq 0 \Longleftrightarrow a_{i i} \neq 0 \quad \forall i=1,2, \ldots, n
$$
Vom considera sistemul liniar cu matrice superior triunghiulară:

\begin{align*}
a_{11} x_{1}+\cdots+a_{1 i} x_{i}+\cdots+a_{1 n-1} x_{n-1}+a_{1 n} x_{n}&=b_{1}\\
\cdots\\
a_{i i} x_{i}+\cdots+a_{i n-1} x_{n-1}+a_{i n} x_{n}&=b_{i}\\
\cdots\\
a_{n-1 n-1} x_{n-1}+a_{n-1 n} x_{n}&=b_{n-1}\\
a_{n n} x_{n}&=b_{n}\\
\end{align*}

Necunoscutele $x_{1}, x_{2}, \ldots, x_{n}$ se deduc pe rând, folosind ecuaţiile sistemului de la ultima către prima. Din ultima ecuaţie, deducem valoarea lui $x_{n}$ :
$$
x_{n}=\frac{b_{n}}{a_{n n}}
$$
Folosind valoarea lui $x_{n}$ calculată mai sus, din penultima ecuaţie obţinem:
$$
x_{n-1}=\frac{b_{n-1}-a_{n-1 n} x_{n}}{a_{n-1 n-1}}
$$


Continuăm să calculăm valori $x_{i}$ din ecuaţiile sistemului:
$$
x_{i}=\frac{b_{i}-a_{i i+1} x_{i+1}-\cdots-a_{i n} x_{n}}{a_{i i}}
$$
Din prima ecuaţie găsim valoarea lui $x_{1}$ :
$$
x_{1}=\frac{b_{1}-a_{12} x_{2}-\cdots-a_{1 n} x_{n}}{a_{11}}
$$
Procedeul descris mai sus poartă numele de metoda substituţiei inverse pentru rezolvarea sistemelor liniare cu matrice superior triunghiulară:
$$
x_{i}=\frac{\left(b_{i}-\sum_{j=i+1}^{n} a_{i j} x_{j}\right)}{a_{i i}} \quad, \quad i=n, n-1, \ldots, 2,1
$$

#### Exemplu pentru metoda substitutiei inverse
$$
\left(\begin{array}{ccc}
3 & 1 & -5 \\
& -2 & 4 \\
& & 2
\end{array}\right)\left(\begin{array}{l}
x_{1} \\
x_{2} \\
x_{3}
\end{array}\right)=\left(\begin{array}{c}
1 \\
10 \\
6
\end{array}\right)$$

#### Rezolvare:

$$x_n = \frac{b_n}{a_{nn}} \Rightarrow x_3 = \frac{b_3}{a_{33}} = \frac{6}{2} = 3$$


$$
x_{i}=\frac{\left(b_{i}-\sum_{j=i+1}^{n} a_{i j} x_{j}\right)}{a_{i i}} \quad, \quad i=n, n-1, \ldots, 2,1
\Rightarrow
x_2=\frac{10 - (4 * 3)}{-2}=1
$$

$$
x_1=\frac{1 - ((1 * 1) + (-5 * 3))}{3} = \frac{1 - (-14)}{3} = \frac{15}{3} = 5
$$

In [365]:
def substitution_method(A, b, n):
    if A[0][0] == 0:
        raise ValueError("The matrix is singular")
    else:
        x = list(0 for j in range(0, n))
        x[n - 1] = b[n - 1] / A[n - 1][n - 1]
        for i in range(n - 2, -1, -1):
            suma = 0
            for j in range(i + 1, n):
                suma = suma + A[i][j] * x[j]
            v = b[i] - suma
            x[i] = v / A[i][i]

        return x

In [366]:
# Generate random superior triangular system
n = 10
A = np.random.randint(low=1, high=100, size=n*n).reshape(n, n)
A = np.triu(A)
b = np.random.randint(low=1, high=100, size=n)

A, b

(array([[10, 16, 65, 29, 90, 94, 30,  9, 74,  1],
        [ 0, 37, 17, 12, 55, 89, 63, 34, 73, 79],
        [ 0,  0, 55, 78, 70, 14, 26, 14, 93, 87],
        [ 0,  0,  0, 13, 66, 32, 58, 37, 28, 19],
        [ 0,  0,  0,  0, 95, 12, 29, 75, 89, 10],
        [ 0,  0,  0,  0,  0, 12, 18, 47,  8, 76],
        [ 0,  0,  0,  0,  0,  0,  6,  5, 72, 89],
        [ 0,  0,  0,  0,  0,  0,  0, 16,  7, 86],
        [ 0,  0,  0,  0,  0,  0,  0,  0, 43, 58],
        [ 0,  0,  0,  0,  0,  0,  0,  0,  0, 24]]),
 array([ 4, 30, 17, 85, 83, 15, 52, 80, 18, 51]))

In [367]:
# Apply substitution method
x = substitution_method(A, b, n)
x

[-138.82179681562053,
 -12.535229963049725,
 40.2585788731872,
 -33.34281160750714,
 4.585178243574051,
 -6.084060077519383,
 10.977107558139537,
 -5.351017441860465,
 -2.447674418604651,
 2.125]

In [368]:
# Compute norm between the matrix multiplication of A and our solution
diff = A @ x - b
diff

array([ 0.,  0.,  0.,  0., -0.,  0.,  0.,  0.,  0.,  0.])

In [369]:
# Test if the matrix is singular
A[2, 2] = 0

try: 
    x = substitution_method(A, b, n)
except Exception as e: 
    print(e)

C:\Users\alexa\AppData\Local\Temp\ipykernel_14264\614015635.py:12: RuntimeWarning: divide by zero encountered in double_scalars
  x[i] = v / A[i][i]
C:\Users\alexa\AppData\Local\Temp\ipykernel_14264\614015635.py:10: RuntimeWarning: invalid value encountered in double_scalars
  suma = suma + A[i][j] * x[j]


### 2. Algoritmul de eliminare Gauss


Ideea de rezolvare este de a transforma succesiv sistemul folosind operaţii elementare (ce nu modifică soluţia sistemului) şi a aduce matricea $A$ la o formă superior triunghiulară . Algoritmul se desfăşoară în $(n-1)$ paşi. La un pas $l$ oarecare se transformă coloana $l$ a matricei $A$ în formă superior triunghiulară fără a modifica forma triunghiulară a primelor $(l-1)$ coloane.


**Pas $l$ :** Presupunem că elementul de pe poziţia $(l, l)$ numit şi element pivot este nenul, $a_{l l} \neq 0$. Pentru $i=l+1, \ldots, n$ se înmulteste linia $l$ a matricei $A$ $\mathrm{cu}\left(-a_{i l} / a_{l l}\right)$ şi se adună la linia $i$. Schimbare se face şi asupra componentei $i$ a vectorului $b$. Matricea $A$ şi vectorul $b$ se modifică astfel:
$$
\begin{aligned}
&a_{i j}^{\prime}=a_{i j}-\frac{a_{i l}}{a_{l l}} a_{l j} \quad, \quad i=\overline{l+1, n}, j=\overline{l+1, n}  &(5)\\
&b_{i}^{\prime}=b_{i}-\frac{a_{i l}}{a_{l l}} b_{l} \quad, \quad i=\overline{l+1, n}  &(6)\\
&a_{i l}^{\prime}=0 \quad, \quad i=\overline{l+1, n}, &(7)\\
&a_{i j}^{\prime}=a_{i j} \quad, \quad b_{i}^{\prime}=b_{i} \text { pentru restul indicilor } i, j
\end{aligned}
$$

In formula $(5)$ factorul $f=\frac{a_{i l}}{a_{l l}}$ se calculează în afara buclei for pentru $j$.

#### Alegerea pivotului $a_{l l} \neq 0\left(\left|a_{l l}\right|>\epsilon\right)$

Pentru a aduce pe poziţia $(l, l)$ un element nenul avem trei posibilităti:
1. **Varianta fără pivotare** : Se alege primul indice $i_{0} \in\{l, l+1, \cdots, n\}$ astfel ca $a_{i_{0} l} \neq 0$. Se interschimbă liniile $i_{0}$ şi $l$ ale matricei $A$ şi componentele $i_{0}$ şi $l$ ale vectorului $b$.
2. **Varianta cu pivotare parțială**: 
Se alege indicele $i_{0} \in\{l, l+1, \cdots, n\}$ astfel ca:
$$
\left|a_{i_{0} l}\right|=\max \left\{\left|a_{i l}\right|, i=\overline{l, n}\right\} .
$$
Se interschimbă liniile $i_{0}$ şi $l$ ale matricei $A$ şi componentele $i_{0}$ şi $l$ ale vectorului $b$.
3. **Varianta cu pivotare totală** 
Se aleg indicii $i_{0}, j_{0} \in\{l, l+1, \cdots, n\}$ astfel ca:
$$
\left|a_{i_{0} j_{0}}\right|=\max \left\{\left|a_{i j}\right|, i=\overline{l, n}, j=\overline{l, n}\right\} .
$$
Se interschimbă liniile $i_{0}$ şi $l$, coloanele $j_{0}$ şi $l$ ale matricei $A$ şi componentele $i_{0}$ şi $l$ ale vectorului $b$.
Dacă după efectuarea pivotării (indiferent de varianta de pivotare aleasă) elementul pivot este nul
$$
a_{l l}=0 \quad \sim \quad \operatorname{abs}(a[l, l]) \leq \epsilon
$$
atunci matricea $A$ este singulară.

**Observaţii:**
1. In pasul Gauss $l$ $(5)+(6)+(7)$ calculele se pot efectua în matricea $A$ iniţială $\left(a^{\prime}=a\right)$.

2. Dacă pentru memorarea matricei $A$ şi a vectorului $b$ se foloseşte o matrice cu $n$ linii şi $(n+1)$ coloane, vectorul $b$ fiind memorat în coloana $(n+1)$ a matricei $A$, calculele $(6)$ sunt incluse în $(5)$ pentru $j=n+1$; se simplifică şi interschimbarea liniilor $i_{0}$ şi $l$.

3. Dacă pivotul se alege folosind varianta a 3-a, cu pivotare totală, la final trebuie să avem grijă sa restabilim ordinea iniţială a componentelor vectorului soluţie (ţinând cont de coloana $j_{0}$ a pivotului de la fiecare pas).

#### Exemplu pentru algoritmul de eliminare Gauss

Youtube: https://www.youtube.com/watch?v=2j5Ic2V7wq4 - puteti cauta si altele pt "Gaussian elimination"

Exemplul din tema (fara pivotare):
$$
A=\left(\begin{array}{lll}
2 & 0 & 1 \\
0 & 2 & 1 \\
4 & 4 & 6
\end{array}\right) x=\left(\begin{array}{c}
5 \\
1 \\
14
\end{array}\right)
$$

1. **Pasul 1:** Verificam daca $a_{11}$ este nenul. Raspunsul este da, deci $a_{11}$ este un pivot valid. Astfel putem incepe sa facem valorile $a_{21}$ si $a_{31}$ nule.
  * $a_{21}$ este deja nul, deci vom alsa linia $2$ neschimbata
  * $a_{31}$ nu este nul, deci incercam sa facem elementul 0 inmultind linia 3 cu valoarea $(−𝑎_{𝑖𝑙}/𝑎{𝑙𝑙})$ unde $i = 3$, asadar Linia 3 devine $Linia3-2 * Linia 1$
  Asadar
  $$
A=\left(\begin{array}{lll}
2 & 0 & 1 \\
0 & 2 & 1 \\
0 & 4 & 4
\end{array}\right) x=\left(\begin{array}{c}
5 \\
1 \\
4
\end{array}\right)
$$

1. **Pasul 2:** Verificam daca $a_{22}$ este nenul. Raspunsul este da, deci $a_{22}$ este un pivot valid. Astfel putem incepe sa facem valorea $a_{23}$ nula.
  * $a_{23}$ nu este nula, deci incercam sa facem elementul 0 inmultind linia 2 cu -4/2 si adunand la linia 3
  Asadar
  $$
A=\left(\begin{array}{lll}
2 & 0 & 1 \\
0 & 2 & 1 \\
0 & 0 & 2
\end{array}\right) x=\left(\begin{array}{c}
5 \\
1 \\
2
\end{array}\right)
$$

In [370]:
def search_pivot(matrix, column):
    max = 0
    index = 0
    for i in range(column, len(matrix[column])):
        if max < abs(matrix[i][column]):
            max = abs(matrix[i][column])
            index = i
    return index


def interschimba_linii(A, b, l, index):
    A[[l, index]] = A[[index, l]]
    aux = b[index]
    b[index] = b[l]
    b[l]=aux
    return A, b

In [385]:
def gauss_algorithm(A, b, n):
    l = 0
    index = search_pivot(A, l)
    A, b = interschimba_linii(A, b, l, index)
    b_prim = b
    a_prim = A
    while l < n - 1 and a_prim[l][l] != 0:
        for i in range(l + 1, n):
            f = a_prim[i][l] / a_prim[l][l]
            for j in range(l + 1, n):
                a_prim[i][j] = a_prim[i][j] - f * a_prim[l][j]
            b_prim[i] = b_prim[i] - f * b_prim[l]
            a_prim[i][l] = 0
        l += 1
        index = search_pivot(a_prim, l)
        a_prim, b_prim = interschimba_linii(a_prim, b_prim, l, index)

    if A[l][l] == 0:
        print("Matrice singulara")
    else:
        x = substitution_method(a_prim, b_prim, n)
        diff = a_prim @ x - b_prim
        for o in diff:
            if int(o) != 0:
                print("Solutie gresita")
            else:
                return a_prim, b_prim, 0

In [380]:
# Generate random superior triangular system
n = 5
A = np.random.randint(low=1, high=100, size=n*n).reshape(n, n)
b = np.random.randint(low=1, high=100, size=n)
A, b

(array([[94, 76, 87, 38, 12],
        [22, 34, 96, 44, 89],
        [97, 74, 41, 44, 91],
        [72,  9, 86, 73, 29],
        [31, 90, 26, 79, 82]]),
 array([86, 63, 14, 42, 34]))

In [381]:
A_gauss, b_gauss, evo = gauss_algorithm(A, b, n=n) # here is presented the first method
A_gauss, b_gauss

(array([[  97,   74,   41,   44,   91],
        [   0,   66,   12,   64,   52],
        [   0,    0,   82,   17,   54],
        [   0,    0,    0,   69,  -43],
        [   0,    0,    0,    0, -118]]),
 array([14, 29, 51, 10, 43]))

In [374]:
# plot_matrix_evolution(evo, "Gauss algorithm matrix evolution")

In [382]:
x_gauss = substitution_method(A_gauss, b_gauss, n=n)
x_gauss

[-0.34115749338648543,
 0.6463678792344967,
 0.878960943257185,
 -0.0821665438467207,
 -0.3644067796610169]

Fie $x_{\text {Gauss }}$ soluţia aproximativă calculată. Să se verifice soluţia afişând următoarea normă:
$$
\left\|A^{i n i t} x_{G a u s s}-b^{i n i t}\right\|_{2}
$$
$A^{\text {init }}$ şi $b^{\text {init }}$ sunt datele iniţiale, nu cele modificate pe parcursul algoritmului. Am notat cu $\|\cdot\|_{2}$ norma Euclidiană. Această normă ar trebui să fie mai mică decât $10^{-6}$, dacă aţi implementat corect metoda eliminării Gauss.

In [393]:
from numpy.linalg import norm, solve, inv
def get_norm(a):
    print(a)
    norm_l2 = norm(a)
    return norm_l2

In [389]:
get_norm(A @ x_gauss - b)

[ 0. -0.  0.  0.  0.]


1.0048591735576161e-14

Folosindu-se una din bibliotecile menţionate în pagina laboratorului, să se calculeze şi să se afişeze soluţia sistemului $A x=b$ şi inversa matricei $A, x_{b i b l}$ şi $A_{b i b l}^{-1}$. Să se afişeze următoarele norme:
$$
\begin{gathered}
\left\|x_{\text {Gauss }}-x_{\text {bibl }}\right\|_{2} \\
\left\|x_{\text {Gauss }}-A_{\text {bibl }}^{-1} b^{i n i t}\right\|_{2} .
\end{gathered}
$$
Aceste norme ar trebui să fie mai mici decât $10^{-6}$.

In [390]:
def lib_solve(A,b):
    return solve(A,b)

In [391]:
x_lib = lib_solve(A, b)
x_lib

array([-0.34115749,  0.64636788,  0.87896094, -0.08216654, -0.36440678])

In [392]:
get_norm(x_gauss - x_lib)

[ 0. -0.  0. -0.  0.]


1.249000902703301e-16

In [394]:
def lib_inv(A):
    return inv(A)

In [395]:
get_norm(x_gauss - lib_inv(A) @ b)

[-0.  0.  0. -0.  0.]


5.721958498152797e-17

### 3. Calculul inversei unei matrice folosind metoda lui Gauss

Dacă se cunoaşte o metodă numerică de rezolvare a sistemelor liniare (în cazul de faţă se va folosi algoritmul de eliminare Gauss), coloanele matricei inverse se pot aproxima rezolvând $n$ sisteme liniare.
Coloana $j$ a matricei $A^{-1}$ se aproximează rezolvând sistemul liniar:
$$
\begin{aligned}
&A x=e_{j}, j=1,2, \ldots, n, \\
&e_{j}=(0, \ldots, 1,0 \ldots, 0)^{T}, 1 \text { este pe poziţia } j \text { în vectorul } e_{j}
\end{aligned}
$$
Procedura de calcul a matricei $A_{\text {Gauss }}^{-1}$ este următoarea:
- Se calculează eliminarea Gauss a matricei extinse $\left[A, I_{n}\right] \in \mathbb{R}^{n \times 2 n}\left(I_{n}\right.$ este matricea unitate de dimensiune $n$ ). Se adaptează varianta a 2-a a algoritmului de eliminare Gauss cu pivotare parţială, astfel încât să modifice toate coloanele matricei $I_{n}$ simultan, împreună cu transformarea matricei $A$ în formă superior triunghiulară. În varianta a 2 -a a algoritmului descrisă mai sus se face transformarea Gauss a matricei $[A, b] \in \mathbb{R}^{n \times(n+1)}$.
La final, matricea va avea următoarea formă $\left[R, I_{\text {transformat }}\right]$ unde $R$ este forma superior triunghiulară a matricei $A$ iar $I_{\text {transformat }}$ este matricea $I_{n}$ modificată conform algoritmului de eliminare Gauss.
- for $j=1, \ldots, n$
  1. $b=$ coloana $j$ a matricei $I_{\text {transformat }}$;
  2. se rezolvă sistemul superior triunghiular $R x=b$, folosind metoda substituţiei inverse, se obţine soluţia $x^{*}\left(x^{*}\right.$ este soluţia sistemului liniar $A x=e_{j}$);
  3. se memorează $x^{*}$ în coloana $j$ a matricei $A_{\text {Gauss }}^{-1}$;

Procedura de mai sus detaliază, în fapt, rezolvarea numerică a ecuaţiei matriceale:

$$A X=I_{n}, X \in \mathbb{R}^{n \times n}, I_{n}=\text { matricea unitate. }$$

In [396]:
def get_inv(A):
    #TODO
    return 0

In [397]:
my_inv = get_inv(A)
my_inv @ A

ValueError: matmul: Input operand 0 does not have enough dimensions (has 0, gufunc core with signature (n?,k),(k,m?)->(n?,m?) requires 1)

Folosind algoritmul de eliminare Gauss, calculaţi o aproximare a inversei acestei matrice, $A_{\text {Gauss }}^{-1}$. Să se afişeze:
$$
\left\|A_{\text {Gauss }}^{-1}-A_{\text {bibl }}^{-1}\right\|
$$
Folosiţi orice normă matriceală este implementată în bibliotecă.

In [ ]:
get_norm(lib_inv(A) - my_inv)

### Bonus (25pt)
: Să se adapteze algoritmul de eliminare Gauss cu pivotare parţială pentru matrice tridiagonale de forma:
$$
A=\left(\begin{array}{cccccccc}
a_{1} & b_{1} & 0 & 0 & \cdots & 0 & 0 & 0 \\
c_{1} & a_{2} & b_{2} & 0 & \cdots & 0 & 0 & 0 \\
0 & c_{2} & a_{3} & b_{3} & \cdots & 0 & 0 & 0 \\
\vdots & & & & & & & \\
0 & 0 & 0 & 0 & \cdots & c_{n-2} & a_{n-1} & b_{n-1} \\
0 & 0 & 0 & 0 & \cdots & 0 & c_{n-1} & a_{n}
\end{array}\right)
$$
Să se calculeze soluţia sistemului $A x=g$. La sfârşitul rulării algoritmului de eliminare Gauss pentru matrice tridiagonală, se ajunge la o matrice de forma:
$$
R=\left(\begin{array}{cccccccc}
d_{1} & e_{1} & f_{1} & 0 & \cdots & 0 & 0 & 0 \\
0 & d_{2} & e_{2} & f_{2} & \cdots & 0 & 0 & 0 \\
\vdots & & & & & & & \\
0 & 0 & 0 & 0 & \cdots & d_{n-2} & e_{n-2} & f_{n-2} \\
0 & 0 & 0 & 0 & \cdots & 0 & d_{n-1} & e_{n-1} \\
0 & 0 & 0 & 0 & \cdots & 0 & 0 & d_{n}
\end{array}\right)
$$
La aplicarea algoritmului de eliminare Gauss, să se lucreze doar cu cei 6 vectori $a, b, c, d, e, f$.

In [ ]:
# To be completed by the brave ones